In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete_1 = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [2]:
#rename the columns
school_data_complete = school_data_complete_1.rename(columns={"Student ID":"student_ID","School ID" : "school_ID"})

## **District Summary**

In [3]:
#Calculate School total figures
school_unique = school_data_complete.school_ID.unique()
school_count = len(school_unique)

#Calcuate total students
student_unique = school_data_complete.student_ID.unique()
student_count = len(student_unique)

#Calculate total budget (needs looking at!!)
budget_unique = school_data_complete.budget.unique()
total_budget = sum(budget_unique)

#Calculate the average scores 
ave_read = round(school_data_complete.reading_score.mean(),2)
ave_math = round(school_data_complete.math_score.mean(),2)

#Calculate passing percentage
pass_read = school_data_complete[school_data_complete["reading_score"] >= 70]
int_read = (len(pass_read))

pass_math = school_data_complete[school_data_complete["math_score"] >= 70]
int_math = (len(pass_math))

per_read = round(((int_read/student_count)*100),2)
per_math = round(((int_math/student_count)*100),2)

#Calculate the passing percentage of both
pass_both = school_data_complete[(school_data_complete.reading_score >=70) & (school_data_complete.math_score >=70) ]
int_both = (len(pass_both))
per_both = round(((int_both/student_count)*100),2)

#create a new df
district_df = pd.DataFrame ({'Total Schools': [school_count],'Total Students': [student_count], 'Total Budget' : [total_budget],
                             'Ave Reading Score' : [ave_read], 'Ave Maths Score' : [ave_math], '% passing Reading': [per_read], 
                             '% passing Maths': [per_math], '% passing Both': [per_both]})

#add the correct format 
district_df["Total Students"] = district_df["Total Students"].map("{:,}".format)
district_df["Total Budget"] = district_df["Total Budget"].map("${:,.2f}".format)
district_df["% passing Reading"] = district_df["% passing Reading"].map("{:,.2f}%".format)
district_df["% passing Maths"] = district_df["% passing Maths"].map("{:,.2f}%".format)
district_df["% passing Both"] = district_df["% passing Both"].map("{:,.2f}%".format)
display(district_df)

,Total Schools,Total Students,Total Budget,Ave Reading Score,Ave Maths Score,% passing Reading,% passing Maths,% passing Both
0,15,"39,170","$24,649,428.00",81.88,78.99,85.81%,74.98%,65.17%


## School Summary

In [4]:
#Create a School Summary Data Frame 
#Group the data by schools
school_df = school_data_complete.groupby(["school_name"])
school_df.count()

#Calculate the means of the scores
scores_df = school_df[["reading_score", "math_score"]].mean()

#Merge in the schools information 
scores2_df = pd.merge(scores_df, school_data, how="left", on=["school_name", "school_name"])

#remove the School ID Colums
scores2_df.drop('School ID', axis=1, inplace=True)

#Calculate the budget per student 
scores2_df["budget_per"] = scores2_df['budget']/scores2_df['size']

In [5]:
#reading data frame 
sread_df = school_data_complete.loc[(school_data_complete['reading_score'] >= 70)]
read_pass_df = pd.DataFrame(sread_df.groupby(['school_name']).size())
read_pass_df.rename(columns={0:'read_pass'},inplace=True)

read_merge_df = pd.merge(read_pass_df,school_data, how="left", on=["school_name","school_name"])
read_merge_df["read_per"] = (read_merge_df["read_pass"]/read_merge_df['size'])*100

In [6]:
#Merge the reading frame
scores2_df=pd.merge(scores2_df,read_merge_df[['school_name','read_per']], how="left", on=['school_name','school_name'])

In [7]:
#math data frame 
smath_df = school_data_complete.loc[(school_data_complete['math_score'] >= 70)]
math_pass_df = pd.DataFrame(smath_df.groupby(['school_name']).size())
math_pass_df.rename(columns={0:'math_pass'},inplace=True)

math_merge_df = pd.merge(math_pass_df,school_data, how="left", on=["school_name","school_name"])
math_merge_df["math_per"] = (math_merge_df["math_pass"]/math_merge_df['size'])*100


In [8]:
#Merge the maths data frame
scores2_df=pd.merge(scores2_df,math_merge_df[['school_name','math_per']], how="left", on=['school_name','school_name'])

In [9]:
#both data frame
sboth_df = school_data_complete.loc[(school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70)]                                 
both_pass_df = pd.DataFrame(sboth_df.groupby(['school_name']).size())
both_pass_df.rename(columns={0:'both_pass'},inplace=True)

both_merge_df = pd.merge(both_pass_df,school_data, how="left", on=["school_name","school_name"])
both_merge_df["both_per"] = (both_merge_df["both_pass"]/both_merge_df['size'])*100

In [10]:
#merege the both data frame
scores2_df=pd.merge(scores2_df,both_merge_df[['school_name','both_per']], how="left", on=['school_name','school_name'])

In [11]:
#reorganise the table
scores3_df = scores2_df[['school_name', 'type', 'size', 'budget', 'budget_per', 'math_score', 'reading_score',
         'math_per', 'read_per', 'both_per']]


scores3_df["budget"] = scores3_df["budget"].map("${:,.2f}".format)
scores3_df["budget_per"] = scores3_df["budget_per"].map("${:,.2f}".format)
scores3_df["reading_score"] = scores3_df["reading_score"].map("{:,.2f}".format)
scores3_df["math_score"] = scores3_df["math_score"].map("{:,.2f}".format)
scores3_df["math_per"] = scores3_df["math_per"].map("{:,.2f}%".format)
scores3_df["read_per"] = scores3_df["read_per"].map("{:,.2f}%".format)
scores3_df["both_per"] = scores3_df["both_per"].map("{:,.2f}%".format)

scores3_df = scores3_df.rename(columns={'school_name': 'School Name', 'type': 'School Type', 'size': 'Total Students',
                                 'budget' : 'Total School Budget' ,'budget_per': 'Per Student Budget',
                                 'math_score' : 'Average Math Score', 'reading_score' : 'Average Reading Score',
                                 'math_per': '% Passing Math' , 'read_per' : '% Passing Reading' , 
                                 'both_per' : '% Overall Passing'})
scores3_df.set_index('School Name')

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,89.23%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


## Highest Performing Schools (By % Overall Passing)

In [12]:
scores3_df.sort_values(by=['% Overall Passing'], inplace =True, ascending =False)
scores4_df = scores3_df.set_index('School Name')
scores4_df.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


## Lowest Performing Schools (By % Overall Passing)¶

In [13]:
scores3_df.sort_values(by=['% Overall Passing'], inplace =True, ascending =True)
scores5_df = scores3_df.set_index('School Name')
scores5_df.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


## Math Score by Grade

In [14]:
#Creates data frames for each grade
grade9 = school_data_complete.loc[(school_data_complete['grade']) == "9th"]
grade9_df = pd.DataFrame(grade9.groupby(['school_name'])['math_score'].mean())

grade10 = school_data_complete.loc[(school_data_complete['grade']) == "10th"]
grade10_df = pd.DataFrame(grade10.groupby(['school_name'])['math_score'].mean())

#Merge the 9th and 10th grade
grade_m1_df=pd.merge(grade9_df, grade10_df[['math_score']], how="left", on=['school_name'])
grade_m1_df.rename(columns={'math_score_x':'9th','math_score_y':'10th'},inplace=True)

#Create the 11th grade data frame
grade11 = school_data_complete.loc[(school_data_complete['grade']) == "11th"]
grade11_df = pd.DataFrame(grade11.groupby(['school_name'])['math_score'].mean())

#Merge the 11th grade with the previous two grades
grade_m2_df=pd.merge(grade_m1_df, grade11_df[['math_score']], how="left", on=['school_name'])
grade_m2_df.rename(columns={'math_score':'11th'},inplace=True)

#Create the 12th grade data frame
grade12 = school_data_complete.loc[(school_data_complete['grade']) == "12th"]
grade12_df = pd.DataFrame(grade12.groupby(['school_name'])['math_score'].mean())

#Merege the 12th grade into the final grades
grade_m3_df=pd.merge(grade_m2_df, grade12_df[['math_score']], how="left", on=['school_name'])
grade_m3_df.rename(columns={'math_score':'12th'},inplace=True)

#Add formatting to the final DataFrame
grade_m3_df["9th"] = grade_m3_df["9th"].map("{:,.2f}".format)
grade_m3_df["10th"] = grade_m3_df["10th"].map("{:,.2f}".format)
grade_m3_df["11th"] = grade_m3_df["11th"].map("{:,.2f}".format)
grade_m3_df["12th"] = grade_m3_df["12th"].map("{:,.2f}".format)
grade_m3_df.index.names = ['School Name']
grade_m3_df

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade

In [15]:
#Creates data frames for each grade
rgrade9 = school_data_complete.loc[(school_data_complete['grade']) == "9th"]
rgrade9_df = pd.DataFrame(rgrade9.groupby(['school_name'])['reading_score'].mean())

rgrade10 = school_data_complete.loc[(school_data_complete['grade']) == "10th"]
rgrade10_df = pd.DataFrame(rgrade10.groupby(['school_name'])['reading_score'].mean())

#Merge the 9th and 10th grade
rgrade_m1_df=pd.merge(rgrade9_df, rgrade10_df[['reading_score']], how="left", on=['school_name'])
rgrade_m1_df.rename(columns={'reading_score_x':'9th','reading_score_y':'10th'},inplace=True)

#Create the 11th grade data frame
rgrade11 = school_data_complete.loc[(school_data_complete['grade']) == "11th"]
rgrade11_df = pd.DataFrame(rgrade11.groupby(['school_name'])['reading_score'].mean())

#Merge the 11th grade with the previous two grades
rgrade_m2_df=pd.merge(rgrade_m1_df, rgrade11_df[['reading_score']], how="left", on=['school_name'])
rgrade_m2_df.rename(columns={'reading_score':'11th'},inplace=True)

#Create the 12th grade data frame
rgrade12 = school_data_complete.loc[(school_data_complete['grade']) == "12th"]
rgrade12_df = pd.DataFrame(rgrade12.groupby(['school_name'])['reading_score'].mean())

#Merege the 12th grade into the final grades
rgrade_m3_df=pd.merge(rgrade_m2_df, rgrade12_df[['reading_score']], how="left", on=['school_name'])
rgrade_m3_df.rename(columns={'reading_score':'12th'},inplace=True)

#Add formatting to the final DataFrame
rgrade_m3_df["9th"] = rgrade_m3_df["9th"].map("{:,.2f}".format)
rgrade_m3_df["10th"] = rgrade_m3_df["10th"].map("{:,.2f}".format)
rgrade_m3_df["11th"] = rgrade_m3_df["11th"].map("{:,.2f}".format)
rgrade_m3_df["12th"] = rgrade_m3_df["12th"].map("{:,.2f}".format)
rgrade_m3_df.index.names = ['School Name']
rgrade_m3_df

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

In [16]:
#Create bins
spend_bin = [0, 585, 630, 645, 680]
spend_labels = ["<$585","$585-630","$630-645","$645-680"]

#Add the budget spent per pupil to the main data frame
school_data_complete["budget_per"] = school_data_complete['budget']/school_data_complete['size']
school_data_complete["Overall_pass"] = (school_data_complete['reading_score']+school_data_complete['math_score'])/2
school_data_complete["Bins"] = pd.cut(school_data_complete["budget_per"], spend_bin, labels=spend_labels)


#Create the data frame grouped by Bins
school_spend = school_data_complete.groupby("Bins")
school_spend_df = school_spend[["reading_score", "math_score"]].mean()


#Merge the reading score by bins
spend_read_df = school_data_complete.loc[(school_data_complete['reading_score'] >= 70)]
spend_read_pass_df = pd.DataFrame(spend_read_df.groupby(['Bins']).size())
spend_read_pass_df.rename(columns={0:'read_pass'},inplace=True)

spend1_df=pd.merge(school_spend_df, spend_read_pass_df, how="left", on=['Bins'])


#Merge the maths score by bins
spend_math_df = school_data_complete.loc[(school_data_complete['math_score'] >= 70)]
spend_math_pass_df = pd.DataFrame(spend_math_df.groupby(['Bins']).size())
spend_math_pass_df.rename(columns={0:'math_pass'},inplace=True)

spend2_df=pd.merge(spend1_df, spend_math_pass_df, how="left", on=['Bins'])


#Merge the both score by bins
spend_both_df = school_data_complete.loc[(school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70)]
spend_both_pass_df = pd.DataFrame(spend_both_df.groupby(['Bins']).size())
spend_both_pass_df.rename(columns={0:'both_pass'},inplace=True)

spend3_df=pd.merge(spend2_df, spend_both_pass_df, how="left", on=['Bins'])

#Add on school size for calculation 
school_size = school_data_complete.groupby("Bins")
school_size_df = school_spend[["size"]].count()
school_size_df
spend4_df=pd.merge(spend3_df, school_size_df, how="left", on=['Bins'])


#Calculate the %
spend4_df["spend_mathsp"] = (spend4_df['math_pass']/spend4_df['size'])*100
spend4_df["spend_readp"] = (spend4_df['read_pass']/spend4_df['size'])*100
spend4_df["spend_bothp"] = (spend4_df['both_pass']/spend4_df['size'])*100


#remove unwated colums
spend4_df.drop(['read_pass','math_pass','both_pass','size'], axis=1, inplace=True)


#rename and format the columns
spend4_df["math_score"] = spend4_df["math_score"].map("{:,.2f}".format)
spend4_df["reading_score"] = spend4_df["reading_score"].map("{:,.2f}".format)
spend4_df["spend_mathsp"] = spend4_df["spend_mathsp"].map("{:,.2f}%".format)
spend4_df["spend_readp"] = spend4_df["spend_readp"].map("{:,.2f}%".format)
spend4_df["spend_bothp"] = spend4_df["spend_bothp"].map("{:,.2f}%".format)
spend4_df = spend4_df.rename(columns={'Bins': 'Spending Ranges (Per Student)','reading_score': 'Average Reading Score',
                                    'math_score' : 'Average Math Score', "spend_mathsp" : "% Passing Math", 
                                    'spend_readp' : '% Passing Reading', 'spend_bothp' : '% Overall passing'})
spend4_df.index.names = ['Spending Ranges (Per Student)']
spend4_df

,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall passing
Spending Ranges (Per Student),,,,,
<$585,83.96,83.36,93.70%,96.69%,90.64%
$585-630,82.31,79.98,79.11%,88.51%,70.94%
$630-645,81.30,77.82,70.62%,82.60%,58.84%
$645-680,81.01,77.05,66.23%,81.11%,53.53%


## Scores by School Size

In [17]:
#Create bins
size_bin = [0, 1000, 2000, 5000]
size_labels = ["Small <1000","Medium (1000-2000)","Large (2000-5000)"]

school_data_complete["Bins_size"] = pd.cut(school_data_complete["size"], size_bin, labels=size_labels)

#Create the data frame grouped by Bins
school_size = school_data_complete.groupby("Bins_size")
school_size_df = school_size[["reading_score", "math_score"]].mean()


#Merge the reading score by bins
size_read_df = school_data_complete.loc[(school_data_complete['reading_score'] >= 70)]
size_read_pass_df = pd.DataFrame(size_read_df.groupby(['Bins_size']).size())
size_read_pass_df.rename(columns={0:'read_pass'},inplace=True)

size1_df=pd.merge(school_size_df, size_read_pass_df, how="left", on=['Bins_size'])


#Merge the maths score by bins
size_math_df = school_data_complete.loc[(school_data_complete['math_score'] >= 70)]
size_math_pass_df = pd.DataFrame(size_math_df.groupby(['Bins_size']).size())
size_math_pass_df.rename(columns={0:'math_pass'},inplace=True)

size2_df=pd.merge(size1_df, size_math_pass_df, how="left", on=['Bins_size'])


#Merge the both score by bins
size_both_df = school_data_complete.loc[(school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70) ]
size_both_pass_df = pd.DataFrame(size_both_df.groupby(['Bins_size']).size())
size_both_pass_df.rename(columns={0:'both_pass'},inplace=True)

size3_df=pd.merge(size2_df, size_both_pass_df, how="left", on=['Bins_size'])

#Add on school size for calculation 
school_size = school_data_complete.groupby("Bins_size")
school_size_df = school_size[["size"]].count()
school_size_df
size4_df=pd.merge(size3_df, school_size_df, how="left", on=['Bins_size'])


#Calculate the %
size4_df["spend_mathsp"] = (size4_df['math_pass']/size4_df['size'])*100
size4_df["spend_readp"] = (size4_df['read_pass']/size4_df['size'])*100
size4_df["spend_bothp"] = (size4_df['both_pass']/size4_df['size'])*100


#remove unwated colums
size4_df.drop(['read_pass','math_pass','both_pass','size'], axis=1, inplace=True)


#rename and format the columns
size4_df["math_score"] = size4_df["math_score"].map("{:,.2f}".format)
size4_df["reading_score"] = size4_df["reading_score"].map("{:,.2f}".format)
size4_df["spend_mathsp"] = size4_df["spend_mathsp"].map("{:,.2f}%".format)
size4_df["spend_readp"] = size4_df["spend_readp"].map("{:,.2f}%".format)
size4_df["spend_bothp"] = size4_df["spend_bothp"].map("{:,.2f}%".format)
size4_df = size4_df.rename(columns={'Bins': 'Spending Ranges (Per Student)','reading_score': 'Average Reading Score',
                                    'math_score' : 'Average Math Score', "spend_mathsp" : "% Passing Math", 
                                    'spend_readp' : '% Passing Reading', 'spend_bothp' : '% Overall passing'})
size4_df.index.names = ['School Size']

size5_df =size4_df[['Average Math Score','Average Reading Score', '% Passing Math','% Passing Reading', '% Overall passing']]

size5_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall passing
School Size,,,,,
Small <1000,83.83,83.97,93.95%,96.04%,90.14%
Medium (1000-2000),83.37,83.87,93.62%,96.77%,90.62%
Large (2000-5000),77.48,81.20,68.65%,82.13%,56.57%


## Scores by school type

In [18]:
#create types DataFrame
school_type_df = pd.DataFrame(school_data_complete.groupby(['type'])[["reading_score", "math_score"]].mean())
school_type_df

#Add maths scores 
type_math_df = school_data_complete.loc[(school_data_complete['math_score'] >= 70)]
type_math_pass_df = pd.DataFrame(type_math_df.groupby(['type']).size())
type_math_pass_df.rename(columns={0:'math_pass'},inplace=True)

school_type2_df=pd.merge(school_type_df, type_math_pass_df, how="left", on=['type'])
school_type2_df

#Add reading scores 
type_read_df = school_data_complete.loc[(school_data_complete['reading_score'] >= 70)]
type_read_pass_df = pd.DataFrame(type_read_df.groupby(['type']).size())
type_read_pass_df.rename(columns={0:'read_pass'},inplace=True)

school_type3_df=pd.merge(school_type2_df, type_read_pass_df, how="left", on=['type'])
school_type3_df

#Add both scores
type_both_df = school_data_complete.loc[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)]
type_both_pass_df = pd.DataFrame(type_both_df.groupby(['type']).size())
type_both_pass_df.rename(columns={0:'both_pass'},inplace=True)

school_type4_df=pd.merge(school_type3_df, type_both_pass_df, how="left", on=['type'])
school_type4_df

#Add the size 
type_size = school_data_complete.groupby("type")
type_size_df = type_size[["size"]].count()
type_size_df
school_type5_df=pd.merge(school_type4_df, type_size_df, how="left", on=['type'])
school_type5_df

#Calculate the %
school_type5_df["type_mathsp"] = (school_type5_df['math_pass']/school_type5_df['size'])*100
school_type5_df["type_readp"] = (school_type5_df['read_pass']/school_type5_df['size'])*100
school_type5_df["type_bothp"] = (school_type5_df['both_pass']/school_type5_df['size'])*100


#remove unwated columns
school_type5_df.drop(['read_pass','math_pass','both_pass','size'], axis=1, inplace=True)
school_type5_df

#rename and format
school_type5_df["math_score"] = school_type5_df["math_score"].map("{:,.2f}".format)
school_type5_df["reading_score"] = school_type5_df["reading_score"].map("{:,.2f}".format)
school_type5_df["type_mathsp"] = school_type5_df["type_mathsp"].map("{:,.2f}%".format)
school_type5_df["type_readp"] = school_type5_df["type_readp"].map("{:,.2f}%".format)
school_type5_df["type_bothp"] = school_type5_df["type_bothp"].map("{:,.2f}%".format)
school_type5_df = school_type5_df.rename(columns={'type': 'School Type','reading_score': 'Average Reading Score',
                                    'math_score' : 'Average Math Score', "type_mathsp" : "% Passing Math", 
                                    'type_readp' : '% Passing Reading', 'type_bothp' : '% Overall passing'})
school_type5_df.index.names = ['School Type']
school_type5_df


,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall passing
School Type,,,,,
Charter,83.90,83.41,93.70%,96.65%,90.56%
District,80.96,76.99,66.52%,80.91%,53.70%
